## Reading outcomes data from the CDSR ##

When generating the 'contrasting evidence' corpus, I realized we could also generate raw outcome results reported in articles describing in trials that are in Cochrane. And indeed we can -- but it's a bit complicated. This notebook walks through a single outcome for an individual trial in an arbitrary review to demonstrate the challenges.

The data below is what the data I have from the CDSR looks like for a *single outcome* in the CDSR. This outcome corresponds to a specific review (*CD000006 v. 8.0 Absorbable suture materials for primary repair of episiotomy and second degree tears.rm5*). Therefore, we have *outcome-level* and *study-level* (meta-)data; the former applies to the outcome in general while the latter contains the actual data on outcomes collected from each study. 

In [7]:
outcome_data_from_CDSR = {'GroupLabel1': 'Fast absorbing', 'GroupLabel2': 'Standard', 'CmpNo': '2', 'Name': 'Long-term pain: pain at 3 months', 'StudyData': [{'CiEnd': '2.125598189076174', 'Events2': '10', 'Name': None, 'Events1': '9', 'Estimable': 'YES', 'Total1': '155', 'Total2': '153', 'StudyId': 'STD-Gemynthe-1996', 'CiStart': '0.3712986009166181', 'EffectSize': '0.8883870967741936', 'Studies': None}, {'CiEnd': '2.378036446853627', 'Events2': '3', 'Name': None, 'Events1': '3', 'Estimable': 'YES', 'Total1': '40', 'Total2': '21', 'StudyId': 'STD-Leroux-2006', 'CiStart': '0.11590444728661693', 'EffectSize': '0.525', 'Studies': None}], 'Events1': '12', 'Estimable': 'YES', 'EffectMeasure': 'RR', 'CmpName': 'Fast-absorbing synthetic versus standard absorbable synthetic material', 'Total2': '174', 'Total1': '195', 'No': '7', 'CiEnd': '1.6671246647549613', 'CiStart': '0.3708205941100722', 'GraphLabel1': 'Favours fast absorbing', 'Events2': '13', 'EffectSize': '0.786259600030416', 'GraphLabel2': 'Favours standard', 'Studies': '2'}

We have lots of these (10k+, each with a bunch of studies). Concretely, here are the fields associated with each outcome stored in the CDSR.

In [8]:
outcome_data_from_CDSR.keys()

['GroupLabel1',
 'GroupLabel2',
 'CmpNo',
 'Name',
 'StudyData',
 'Events1',
 'Estimable',
 'EffectMeasure',
 'CmpName',
 'Total2',
 'Total1',
 'No',
 'CiEnd',
 'CiStart',
 'GraphLabel1',
 'Events2',
 'EffectSize',
 'GraphLabel2',
 'Studies']

In [3]:
outcome_data_from_CDSR['Name']

'Long-term pain: pain at 3 months'

In [4]:
outcome_data_from_CDSR['CmpName']

'Fast-absorbing synthetic versus standard absorbable synthetic material'

Then there is the *study-level* data associated with each outcome/review. 

In [5]:
len(outcome_data_from_CDSR["StudyData"])

2

So this is a list of two data for two studies (either a very small review or not much evidence for this specific outcome!). In any case, we can pull out one study, which is where we finally get to the raw data:

In [6]:
one_study = outcome_data_from_CDSR["StudyData"][1]
one_study

{'CiEnd': '2.378036446853627',
 'CiStart': '0.11590444728661693',
 'EffectSize': '0.525',
 'Estimable': 'YES',
 'Events1': '3',
 'Events2': '3',
 'Name': None,
 'Studies': None,
 'StudyId': 'STD-Leroux-2006',
 'Total1': '40',
 'Total2': '21'}

Doing some leg work to link the (internal) *StudyId* field to a PMID, we can figure out that this corresponds to PMID 16731075 -- https://www.ncbi.nlm.nih.gov/pubmed/?term=16731075; title "Impact of chromic catgut versus polyglactin 910 versus fast-absorbing polyglactin 910 sutures for perineal repair: A randomized, controlled trial". Here's the abstract. 

![caption](images/abstract.png)

You will notice that the outcome of interest does not seem to be reported in the abstract at all. I retrieved the full paper and went through. The first results table looks like this.

![caption](images/outcomes-table-1.png)

Recall from above that the outcome name is 'Long-term pain: pain at 3 months' and the *comparison* name is 'Fast-absorbing synthetic versus standard absorbable synthetic material'. So then I think the two rightmost columns are what we're after. But none of the numbers under pain *quite* match; the visual analog scale is the closest but off by a 0.5. The key is that the outcome is pain at *3 months*; so this isn't the right table (36-48 hours). 

Scrolling further I found this table:

![caption](images/outcomes-table-2.png)

Bingo. This is at three months, and the 'persistent perineal pain' numbers match the counts reported in the CDSR (3 events in both arms). 

So then some observations:

1. The reported data not always (and, I think usually) not available in the abstract. Assessing the extent to which this is the case requires more work. 

2. Even once we have the full text, figuring out where the extracted results come from takes quite a bit of sluething. That said, it certainly calls for interesting NLP!